# Generate Technical Indicators
---

### Import Libraries

In [1]:
import os
import requests 
import pandas_datareader as pdr

import functools
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import seaborn as sns
import warnings

from sklearn import preprocessing 

from importlib import reload
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 227

### Import Data

In [2]:
ticker = 'TSLA'

In [3]:
stock_df = pdr.get_data_yahoo(ticker, '1980')
# stock_df.to_csv('data/'+csv_title+'.csv')
stock_df.to_csv('data/raw_stocks/'+ticker.upper()+'.csv')

In [4]:
stock_df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-03-04,855.650024,825.159973,849.099976,838.289978,22333200.0,838.289978
2022-03-07,866.140015,804.570007,856.299988,804.580017,24164700.0,804.580017
2022-03-08,849.989990,782.169983,795.530029,824.400024,26799700.0,824.400024
2022-03-09,860.559998,832.010010,839.479980,858.969971,19672600.0,858.969971
2022-03-10,854.450012,810.530029,851.450012,838.299988,19549548.0,838.299988


In [5]:
stock_df.describe()

,High,Low,Open,Close,Volume,Adj Close
count,2946.000000,2946.000000,2946.000000,2946.000000,2.946000e+03,2946.000000
mean,139.170413,132.967921,136.193572,136.209102,3.132607e+07,136.209102
std,252.315411,240.454568,246.717519,246.619960,2.802840e+07,246.619960
min,3.326000,2.996000,3.228000,3.160000,5.925000e+05,3.160000
25%,19.854500,19.022500,19.401000,19.488999,1.307290e+07,19.488999
50%,47.283001,45.720999,46.579000,46.469999,2.488680e+07,46.469999
75%,68.993999,66.728001,67.939503,67.767002,3.974662e+07,67.767002
max,1243.489990,1217.000000,1234.410034,1229.910034,3.046940e+08,1229.910034


### Check for missing data

In [6]:
print('No Missing Data') if sum(stock_df.isna().sum())==0 else stock_df.isna().sum()

No Missing Data


---
# Technical Indicator Calculations  
---

### Momentum

In [7]:
def momentum(close, days):
    m = [None for i in range(days)]
    for i in range(len(close) - days):
        end = i + days
        m.append(close[i] - days)
    return m 

### Relative Strength Index (RSI)

In [8]:
def gain(x):
    if x > 0:
        return x
    else:
        return 0 
        
def loss(x):
    if x < 0:
        return abs(x)
    else:
        return 0 

    
def rsi(stock):
    rsi_list = [None for i in range(14)]
    stock = stock.Change
    
    # Calculate Relative Strength (RS) for first 15 
    avg_gain = sum([i for i in stock[1:15] if i > 0])/14
    avg_loss = sum([abs(i) for i in stock[1:15] if i < 0])/14 
    rs = avg_gain/avg_loss
    # Calculate Relative Strength Index (RSI) for first 15
    rsi = 100 - (100/(1+rs))
    rsi_list.append(rsi)
    
    # Calculate Relative Strength(RS) for 15+ 
    for i in range(15, len(stock)):
        avg_gain = (avg_gain * 13 + gain(stock[i]))/14
        avg_loss = (avg_loss * 13 + loss(stock[i]))/14
        rs = avg_gain/avg_loss
        
        # Calculate Relative Strength Index (RSI) for 15+ 
        rsi = 100 - (100/(1+rs))
        rsi_list.append(rsi)
    
    return rsi_list

### Moving Average Convergence/Divergence (MACD)

In [9]:
def macd(stock):
    exp1 = stock.Close.ewm(span=12, adjust=False).mean()
    exp2 = stock.Close.ewm(span=26, adjust=False).mean()
    ma_con_div = exp1-exp2
    signal = ma_con_div.ewm(span=9, adjust=False).mean()
    return ma_con_div, signal

### Bollinger Bands

In [10]:
def bollinger(stock, window=21):
    roll_mean = stock.Close.rolling(window).mean()
    roll_std = stock.Close.rolling(window).std()
    upper_band = roll_mean + (roll_std*2)
    lower_band = roll_mean - (roll_std*2)
    return upper_band, lower_band

---
# Generating Technical Indicators 
---

In [11]:
files = os.listdir('data/raw_stocks')
stocks = {}

for file in files:
    name = file.lower().split(".")[0]
    stocks[name] = pd.read_csv('data/raw_stocks/'+file)
    
    # Date Feature
    stocks[name]['Date'] = pd.to_datetime(stocks[name]['Date'])
    stocks[name].set_index('Date', inplace = True)
    # Return Feature 
    stocks[name]['Return'] = round(stocks[name]['Close']/stocks[name]['Open']-1, 3)
    # Change Feature: Change previous day's price by absolute value
    stocks[name]['Change'] = (stocks[name].Close - stocks[name].Close.shift(1)).fillna(0)
    # Volatility Feature
    stocks[name]['Volatility'] = stocks[name].Close.ewm(21).std()
    # Moving Average: 7 Days
    stocks[name]['MA7'] = stocks[name].Close.rolling(window=7).mean()
    # Moving Average: 21 Days
    stocks[name]['MA21'] = stocks[name].Close.rolling(window=21).mean()
    
    # Momentum
    stocks[name]['Momentum'] = momentum(stocks[name].Close, 3)
    # Relative Strength Index (RSI)
    stocks[name]['RSI'] = rsi(stocks[name])
    # Moving Average Convergence/Divergence (MACD)
    stocks[name]['MACD'], stocks[name]['Signal'] = macd(stocks[name])
    # Bollinger Bands
    stocks[name]['Upper_Band'], stocks[name]['Lower_Band'] = bollinger(stocks[name])
    
    # Saving
    # stocks[name].dropna(inplace=True)
    stocks[name].to_csv('data/stocks/'+name+'.csv')

In [12]:
stocks[ticker.lower()].tail()

,High,Low,Open,Close,Volume,Adj Close,Return,Change,Volatility,MA7,MA21,Momentum,RSI,MACD,Signal,Upper_Band,Lower_Band
Date,,,,,,,,,,,,,,,,,
2022-03-04,855.650024,825.159973,849.099976,838.289978,22333200.0,838.289978,-0.013,-1.000000,94.699439,843.272853,870.654285,861.369995,43.735022,-29.150511,-33.491684,962.766096,778.542474
2022-03-07,866.140015,804.570007,856.299988,804.580017,24164700.0,804.580017,-0.060,-33.709961,95.242887,843.817139,866.532381,876.890015,40.065681,-30.866360,-32.966619,962.461813,770.602948
2022-03-08,849.989990,782.169983,795.530029,824.400024,26799700.0,824.400024,0.036,19.820007,94.665476,845.892857,861.821905,836.289978,43.089008,-30.277849,-32.428865,955.733162,767.910648
2022-03-09,860.559998,832.010010,839.479980,858.969971,19672600.0,858.969971,0.023,34.569946,92.957422,844.255711,859.518569,835.289978,48.014725,-26.714002,-31.285893,951.084330,767.952808
2022-03-10,854.450012,810.530029,851.450012,838.299988,19549548.0,838.299988,-0.015,-20.669983,91.782127,840.531424,855.532854,801.580017,45.480061,-25.266269,-30.081968,942.864564,768.201145
